<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/dev/RF_Instability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest Instability Analysis

This notebook uses a Random Forest model (Mayer et al., The Cryosphere, 2022) to calculate probability of instability (P_unstable) from SNOWPACK .pro files. This provides additional avalanche risk assessment beyond the standard SNOWPACK output.

**Reference**: [Mayer et al. (2022) - A random forest model to assess snow instability from simulated snow stratigraphy](https://tc.copernicus.org/articles/16/4593/2022/tc-16-4593-2022.pdf)

> **⚠️ Note for Colab Users**: Google Colab uses Python 3.10+ by default, which may have compatibility issues with this model (trained on Python 3.7). The notebook includes automatic compatibility handling, but if you encounter errors, consider running locally with Python 3.9 for best results.

## Features
- Upload your own .pro files or use files from SNOWPACK simulations
- Analyze single profiles for probability of instability
- Generate time series analysis of instability evolution
- Export daily summary statistics to CSV

In [1]:
# @title Install Python dependencies for RF model
# @markdown Installs scikit-learn and other dependencies needed for the Random Forest instability model
# @markdown 
# @markdown **Note for Colab users**: Colab typically uses Python 3.10+. The compatibility layer in this notebook handles sklearn version differences automatically.

import sys
import platform
import os

# Detect if we're in Colab
IN_COLAB = os.path.exists("/content")

print("="*70)
print("ENVIRONMENT SETUP")
print("="*70)
print(f"Python version: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")
print(f"Environment: {'Google Colab' if IN_COLAB else 'Local/Jupyter'}")
print("="*70)
print()

# Install dependencies
print("Installing dependencies...")
!pip -q install --upgrade pip
!pip -q install numpy pandas matplotlib joblib scikit-learn==0.22.2

import sklearn

print()
print("="*70)
print("INSTALLATION COMPLETE")
print("="*70)
print(f"scikit-learn version: {sklearn.__version__}")
print(f"Python version: {sys.version.split()[0]}")
print()

# Check compatibility
python_version = sys.version_info
if IN_COLAB:
    if python_version.major == 3 and python_version.minor >= 10:
        print("⚠ Colab Environment Detected")
        print("  - Python 3.10+ detected (Colab default)")
        print("  - Compatibility layer will handle sklearn version differences")
        print("  - If model loading fails, the notebook will provide detailed guidance")
    else:
        print("✓ Compatible Python version detected")
else:
    if python_version.major == 3 and python_version.minor >= 10:
        print("⚠ Python 3.10+ detected")
        print("  - For best compatibility, consider using Python 3.9")
        print("  - Compatibility layer will attempt to handle differences")
    elif python_version.major == 3 and python_version.minor == 9:
        print("✓ Python 3.9 detected - optimal compatibility")
    elif python_version.major == 3 and python_version.minor <= 8:
        print("✓ Compatible Python version detected")

print("="*70)
print("✓ Dependencies installed successfully")
print("="*70)


ENVIRONMENT SETUP
Python version: 3.12.7
Platform: macOS-10.16-x86_64-i386-64bit
Environment: Local/Jupyter

Installing dependencies...
ERROR: Could not find a version that satisfies the requirement scikit-learn==0.22.2 (from versions: 0.9, 0.10, 0.11, 0.12, 0.12.1, 0.13, 0.13.1, 0.14, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.17, 0.17.1, 0.18, 0.18.1, 0.18.2, 0.19.0, 0.19.1, 0.19.2, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.20.4, 0.21.1, 0.21.2, 0.21.3, 0.22, 0.22.1, 0.22.2.post1, 0.23.0, 0.23.1, 0.23.2, 0.24.0, 0.24.1, 0.24.2, 1.0, 1.0.1, 1.0.2, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.2.0rc1, 1.2.0, 1.2.1, 1.2.2, 1.3.0rc1, 1.3.0, 1.3.1, 1.3.2, 1.4.0rc1, 1.4.0, 1.4.1.post1, 1.4.2, 1.5.0rc1, 1.5.0, 1.5.1, 1.5.2, 1.6.0rc1, 1.6.0, 1.6.1, 1.7.0rc1, 1.7.0, 1.7.1, 1.7.2)
ERROR: No matching distribution found for scikit-learn==0.22.2

INSTALLATION COMPLETE
scikit-learn version: 1.5.1
Python version: 3.12.7

⚠ Python 3.10+ detected
  - For best compatibility, consider using Python 3.9
  - Compatibility

In [2]:
# @title Patch plt_RF.py and load the RF model
# @markdown Fixes syntax warnings and loads the Random Forest model with multiple compatibility strategies

import sys
import pathlib
import joblib
import warnings
import importlib
import re

# Determine RF directory based on environment
if os.path.exists("/content"):
    rf_dir = pathlib.Path("/content/rf_instability").resolve()
else:
    rf_dir = pathlib.Path("./rf_instability").resolve()

if str(rf_dir) not in sys.path:
    sys.path.insert(0, str(rf_dir))

# Fix syntax warnings in plt_RF.py more thoroughly
def patch_plt_RF():
    """Patch plt_RF.py to fix invalid escape sequences"""
    plt_RF_path = rf_dir / 'plt_RF.py'
    if plt_RF_path.exists():
        try:
            with open(plt_RF_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Fix invalid escape sequences using regex to catch all variations
            # Pattern: $...\m...$ (mathregular or mathrm)
            content = re.sub(r'\$([^$]*?)\\([mM])([^$]*?)\$', r'$\1\\\2\3$', content)
            
            # Also fix specific known patterns
            content = content.replace('\\mathregular', r'\mathregular')
            content = content.replace('\\mathrm', r'\mathrm')
            content = content.replace('$P_\\mathrm{unstable}$', r'$P_\mathrm{unstable}$')
            content = content.replace('$\\mathregular{P_{unstable}}$', r'$\mathregular{P_{unstable}}$')
                
            with open(plt_RF_path, 'w', encoding='utf-8') as f:
                f.write(content)
            print("✓ Patched plt_RF.py to fix syntax warnings")
            return True
        except Exception as e:
            print(f"⚠ Could not patch plt_RF.py: {e}")
            return False
    return False

# Patch plt_RF.py before importing
patch_plt_RF()

import numpy as np
import sklearn
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import matplotlib.pyplot as plt

try:
    import get_RF
    import readProfile
    import plt_RF
    print("✓ Helper modules imported")
except ImportError as e:
    print(f"⚠ Error importing helper modules: {e}")
    print("Make sure the RF model download cell ran successfully.")
    raise

MODEL_PATH = rf_dir / 'RF_instability_model.sav'
feature_names = ['viscdefrate','rcflat','sphericity','grainsize','penetrationdepth','slab_rhogs']

def load_rf_model_compat(model_path):
    """
    Load RF model with multiple compatibility strategies.
    
    Strategy 1: Try standard joblib.load (should work with sklearn 0.22.2)
    Strategy 2: Monkey-patch sklearn tree module to accept old format
    Strategy 3: Patch tree nodes after loading
    """
    import sklearn
    from sklearn.tree import _tree
    
    # Strategy 1: Try standard load first
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model = joblib.load(model_path)
        
        # Verify nodes are compatible
        if hasattr(model, 'estimators_'):
            for estimator in model.estimators_:
                if hasattr(estimator, 'tree_'):
                    tree = estimator.tree_
                    if hasattr(tree, 'nodes') and tree.nodes is not None:
                        if 'missing_go_to_left' not in tree.nodes.dtype.names:
                            # Need to patch
                            break
            else:
                # All trees are compatible
                return model
        
        # If we get here, need to patch nodes
        print("⚠ Patching tree nodes for compatibility...")
        for estimator in model.estimators_:
            if hasattr(estimator, 'tree_'):
                tree = estimator.tree_
                if hasattr(tree, 'nodes') and tree.nodes is not None:
                    old_nodes = tree.nodes
                    if 'missing_go_to_left' not in old_nodes.dtype.names:
                        new_dtype = np.dtype([
                            ('left_child', '<i8'),
                            ('right_child', '<i8'),
                            ('feature', '<i8'),
                            ('threshold', '<f8'),
                            ('impurity', '<f8'),
                            ('n_node_samples', '<i8'),
                            ('weighted_n_node_samples', '<f8'),
                            ('missing_go_to_left', 'u1')
                        ])
                        new_nodes = np.zeros(len(old_nodes), dtype=new_dtype)
                        for field in old_nodes.dtype.names:
                            new_nodes[field] = old_nodes[field]
                        new_nodes['missing_go_to_left'] = 0
                        tree.nodes = new_nodes
        
        return model
        
    except ValueError as e:
        if "incompatible dtype" in str(e) or "missing_go_to_left" in str(e):
            # Strategy 2: Monkey-patch sklearn's tree module
            print("⚠ Standard load failed, trying monkey-patch approach...")
            
            original_check = getattr(_tree, '_check_node_ndarray', None)
            
            def patched_check_node_ndarray(node_array, *args, **kwargs):
                """Patch to accept old format nodes (supports sklearn >=1.4 signature)."""
                expected_dtype = kwargs.get('expected_dtype')
                if expected_dtype is None and args:
                    expected_dtype = args[0]

                if node_array.dtype.names and 'missing_go_to_left' not in node_array.dtype.names:
                    new_dtype = np.dtype([
                        ('left_child', '<i8'),
                        ('right_child', '<i8'),
                        ('feature', '<i8'),
                        ('threshold', '<f8'),
                        ('impurity', '<f8'),
                        ('n_node_samples', '<i8'),
                        ('weighted_n_node_samples', '<f8'),
                        ('missing_go_to_left', 'u1')
                    ])

                    new_nodes = np.zeros(node_array.shape, dtype=new_dtype)
                    for field in node_array.dtype.names:
                        new_nodes[field] = node_array[field]
                    new_nodes['missing_go_to_left'] = 0
                    node_array = new_nodes

                if expected_dtype is not None and node_array.dtype != expected_dtype:
                    try:
                        node_array = node_array.astype(expected_dtype)
                    except TypeError:
                        pass

                if original_check:
                    return original_check(node_array, *args, **kwargs)
                return node_array
            
            # Apply monkey-patch
            _tree._check_node_ndarray = patched_check_node_ndarray
            
            try:
                with warnings.catch_warnings():
                    warnings.filterwarnings('ignore', category=UserWarning)
                    model = joblib.load(model_path)
                
                # Patch all tree nodes to ensure compatibility
                if hasattr(model, 'estimators_'):
                    for estimator in model.estimators_:
                        if hasattr(estimator, 'tree_'):
                            tree = estimator.tree_
                            if hasattr(tree, 'nodes') and tree.nodes is not None:
                                old_nodes = tree.nodes
                                if 'missing_go_to_left' not in old_nodes.dtype.names:
                                    new_dtype = np.dtype([
                                        ('left_child', '<i8'),
                                        ('right_child', '<i8'),
                                        ('feature', '<i8'),
                                        ('threshold', '<f8'),
                                        ('impurity', '<f8'),
                                        ('n_node_samples', '<i8'),
                                        ('weighted_n_node_samples', '<f8'),
                                        ('missing_go_to_left', 'u1')
                                    ])
                                    new_nodes = np.zeros(len(old_nodes), dtype=new_dtype)
                                    for field in old_nodes.dtype.names:
                                        new_nodes[field] = old_nodes[field]
                                    new_nodes['missing_go_to_left'] = 0
                                    tree.nodes = new_nodes
                
                return model
            finally:
                # Restore original function
                if original_check:
                    _tree._check_node_ndarray = original_check
        else:
            raise

try:
    model = load_rf_model_compat(MODEL_PATH)
    print(f"✓ Loaded RF model: {MODEL_PATH.name}")
    
    if not hasattr(model, "estimator"):
        model.estimator = DecisionTreeClassifier(
            criterion=model.criterion,
            max_depth=model.max_depth,
            min_samples_split=model.min_samples_split,
            min_samples_leaf=model.min_samples_leaf,
            min_weight_fraction_leaf=model.min_weight_fraction_leaf,
            max_features=model.max_features,
            max_leaf_nodes=model.max_leaf_nodes,
            min_impurity_decrease=model.min_impurity_decrease,
            random_state=model.random_state,
            splitter="best",
            class_weight=model.class_weight,
            ccp_alpha=getattr(model, "ccp_alpha", 0.0),
        )
    # Colab-specific success message
    import os
    if os.path.exists("/content"):
        print("\n" + "="*70)
        print("✅ SUCCESS - Model loaded in Colab environment")
        print("="*70)
        print("The compatibility layer successfully handled the sklearn version differences.")
        print("You can now proceed with analyzing your .pro files.")
        print("="*70)

except Exception as e:
    print("❗ Model load failed with all compatibility strategies.")
    print("\n" + "="*70)
    print("COMPATIBILITY ISSUE EXPLANATION")
    print("="*70)
    import sklearn
    import os
    
    IN_COLAB = os.path.exists("/content")
    python_version = sys.version_info
    
    if IN_COLAB:
        colab_note = """
⚠️ COLAB ENVIRONMENT DETECTED

Colab uses Python 3.10+ by default, which has stricter pickle protocols
than Python 3.7 (where the model was trained). The compatibility layer
attempted multiple strategies but all failed.

RECOMMENDED SOLUTION FOR COLAB:
"""
        solutions = """
1. RESTART COLAB RUNTIME WITH PYTHON 3.9 (if available)
   - Runtime → Change runtime type → Python version → 3.9
   - Re-run all cells
   - This provides the best compatibility

2. USE LOCAL ENVIRONMENT (RECOMMENDED FOR RELIABILITY)
   - Install locally: conda create -n rf_model python=3.9 scikit-learn=0.22.2
   - Or use Python 3.8: conda create -n rf_model python=3.8 scikit-learn=0.22.2
   - More reliable than Colab's Python 3.10+

3. DOWNLOAD MODEL AND USE LOCAL JUPYTER
   - Download the notebook and model files
   - Run in local environment with Python 3.9
   - Best long-term solution

4. WAIT FOR MODEL UPDATE
   - Contact maintainers for skops-compatible version
   - Repository: https://code.wsl.ch/mayers/random_forest_snow_instability_model

5. USE COMPATIBILITY CONVERSION (if you have access to Python 3.7/3.9)
   - Convert model to skops format in compatible environment
   - Then load in Colab using: pip install skops; model = skops.load('model.skops')
"""
    else:
        colab_note = ""
        solutions = """
SOLUTION OPTIONS (ranked by preference):

1. USE PYTHON 3.8 OR 3.9 (RECOMMENDED)
   - Python 3.8/3.9 + sklearn 0.22.2 generally works better than 3.10+
   - Less strict pickle protocol, better compatibility
   - Command: conda create -n rf_model python=3.9 scikit-learn=0.22.2

2. USE EXACT ORIGINAL ENVIRONMENT
   - Python 3.7 + sklearn 0.22.1 (exact match)
   - Command: conda create -n rf_model python=3.7 scikit-learn=0.22.1

3. CONVERT MODEL TO SKOPS FORMAT (requires original environment)
   - Install skops in original environment: pip install skops
   - Convert: import skops; skops.dump(model, 'model.skops')
   - Load in any environment: model = skops.load('model.skops')

4. CONTACT MODEL MAINTAINERS
   - Request updated model file compatible with newer sklearn
   - Repository: https://code.wsl.ch/mayers/random_forest_snow_instability_model

5. RETRAIN MODEL (if training data available)
   - Retrain with current sklearn version for full compatibility
"""
    
    error_msg = f"""
The RF model was saved with scikit-learn 0.22.1 (Python 3.7) which uses
an older tree node structure without the 'missing_go_to_left' field.

Newer sklearn versions (even 0.22.2 on Python 3.10+) expect this field during
unpickling, causing a dtype mismatch error.
{colab_note}{solutions}

Current environment:
  - Environment: {'Google Colab' if IN_COLAB else 'Local/Jupyter'}
  - Python: {sys.version.split()[0]} ({python_version.major}.{python_version.minor})
  - scikit-learn: {sklearn.__version__}
  - Compatibility strategies attempted: All (standard load, monkey-patch, node patching)
    """
    print(error_msg)
    print("="*70)
    print(f"\nOriginal error:\n{e}")
    print("\n" + "="*70)
    if IN_COLAB:
        print("\n💡 TIP: For best results in Colab, consider:")
        print("   1. Using a local Python 3.9 environment")
        print("   2. Converting the model to skops format")
        print("   3. Contacting model maintainers for updated version")
    raise


⚠ Error importing helper modules: No module named 'get_RF'
Make sure the RF model download cell ran successfully.


ModuleNotFoundError: No module named 'get_RF'

## Troubleshooting: Model Loading Issues

If you encounter model loading errors in Colab, try these solutions:

**Option 1: Restart Runtime**
- Runtime → Restart runtime
- Re-run all cells
- Sometimes clears Python caching issues

**Option 2: Use Local Environment (Recommended)**
```bash
# Create environment
conda create -n rf_model python=3.9 scikit-learn=0.22.2 numpy pandas matplotlib joblib
conda activate rf_model

# Install Jupyter
pip install jupyter notebook

# Run notebook
jupyter notebook RF_Instability_Analysis.ipynb
```

**Option 3: Use Python 3.9 Runtime (if available)**
- Runtime → Change runtime type
- Set Python version to 3.9
- Re-run all cells

**Option 4: Download and Run Locally**
- File → Download → Download .ipynb
- Run in local Python 3.9 environment
- More reliable than Colab for this model


## Upload .pro File

Upload your SNOWPACK .pro file below. You can:
- Upload a file from your computer (Colab)
- Provide a path to a file already in the environment
- Use a file from a SNOWPACK simulation output

In [ ]:
# @title Upload or specify .pro file
# @markdown Choose how to provide your .pro file

import os
import glob
from pathlib import Path

# @markdown ### File Input Method
file_input_method = "path"  # @param ["upload", "path", "glob"]

# @markdown ### If using "path", specify the file path:
file_path = "../input_example/WFJ2_2017.pro"  # @param {type:"string"}

# @markdown ### If using "glob", specify a pattern (e.g., "./output/*.pro"):
glob_pattern = ""  # @param {type:"string"}

# Handle file upload/selection
pro_file = None

if file_input_method == "upload":
    try:
        from google.colab import files
        uploaded = files.upload()
        if uploaded:
            # Get the first uploaded file
            pro_file = list(uploaded.keys())[0]
            print(f"✓ Uploaded file: {pro_file}")
        else:
            print("⚠ No file uploaded")
    except ImportError:
        print("⚠ File upload not available (not in Colab). Use 'path' or 'glob' method instead.")
        print("You can drag and drop files in Jupyter Lab, or use the 'path' method.")

elif file_input_method == "path":
    if file_path and os.path.exists(file_path):
        pro_file = file_path
        print(f"✓ Using file: {pro_file}")
    else:
        print(f"⚠ File not found: {file_path}")

elif file_input_method == "glob":
    if glob_pattern:
        matches = glob.glob(glob_pattern)
        if matches:
            pro_file = matches[0]
            print(f"✓ Found {len(matches)} file(s), using: {pro_file}")
            if len(matches) > 1:
                print(f"  Other matches: {matches[1:]}")
        else:
            print(f"⚠ No files found matching: {glob_pattern}")
    else:
        print("⚠ No glob pattern specified")

if pro_file and os.path.exists(pro_file):
    print(f"\n✓ Ready to analyze: {pro_file}")
    file_size = os.path.getsize(pro_file) / (1024 * 1024)  # Size in MB
    print(f"  File size: {file_size:.2f} MB")
else:
    print("\n⚠ No valid .pro file selected. Please upload or specify a file.")

## Single Profile Analysis

Analyze a single profile from your .pro file for probability of instability at a specific date and time.

In [ ]:
# @title Single Profile: Calculate P_unstable
# @markdown Analyze a single SNOWPACK profile for probability of instability

import datetime

# @markdown ## Analysis Parameters
slope_angle = 0  # @param {type:"number", min:0, max:90}
# @markdown Date/time to analyze (format: YYYY-MM-DD HH:MM)
analysis_date = "2017-02-01 11:00"  # @param {type:"string"}

if not pro_file or not os.path.exists(pro_file):
    print("⚠ No .pro file selected. Please run the upload cell first.")
else:
    # Parse date
    try:
        timestamp = datetime.datetime.strptime(analysis_date, "%Y-%m-%d %H:%M")
    except ValueError:
        print(f"⚠ Date format error. Using default: 2017-02-01 11:00")
        timestamp = datetime.datetime(2017, 2, 1, 11, 0)

    # Read profile and calculate P_unstable
    try:
        prof = readProfile.read_profile(pro_file, timestamp, remove_soil=True)
        df_prof = get_RF.create_RFprof(prof, slope_angle, model)
        
        # Quick sanity check: probabilities within [0,1]
        assert df_prof['P_unstable'].between(0, 1).all(), "P_unstable values must be between 0 and 1"
        
        print(f"✓ Profile loaded and analyzed")
        print(f"  File: {os.path.basename(pro_file)}")
        print(f"  Date: {timestamp}")
        print(f"  Slope angle: {slope_angle}°")
        print(f"  Max P_unstable: {df_prof['P_unstable'].max():.3f}")
        print(f"  Depth at max P_unstable: {df_prof.loc[df_prof['P_unstable'].idxmax(), 'layer_top']:.2f} m")
        
        # Plot
        fig, ax = plt.subplots(figsize=(5, 6))
        plt_RF.plot_sp_single_P0(fig, ax, df_prof, var='P_unstable', colorbar=True)
        plt.title(f"P_unstable Analysis\n{os.path.basename(pro_file)} - {timestamp.strftime('%Y-%m-%d %H:%M')}")
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error analyzing profile: {e}")
        print(f"\nTroubleshooting:")
        print(f"  - Check that the date '{timestamp}' exists in the .pro file")
        print(f"  - Verify the .pro file format is correct")
        print(f"  - Try a different date from the file")
        import traceback
        traceback.print_exc()

## Time Series Analysis

Analyze how P_unstable evolves over time for a seasonal period.

In [ ]:
# @title Time Series: Daily Evolution of P_unstable
# @markdown Analyze how P_unstable evolves over time for a seasonal period

import datetime

# @markdown ## Time Period
year = 2017  # @param {type:"integer"}
start_month = 12  # @param {type:"integer", min:1, max:12}
start_day = 1  # @param {type:"integer", min:1, max:31}
end_month = 4  # @param {type:"integer", min:1, max:12}
end_day = 1  # @param {type:"integer", min:1, max:31}

# @markdown ## Analysis Parameters
slope_angle_ts = 0  # @param {type:"number", min:0, max:90}

if not pro_file or not os.path.exists(pro_file):
    print("⚠ No .pro file selected. Please run the upload cell first.")
else:
    # Create date range
    start = datetime.datetime(year-1 if start_month == 12 else year, start_month, start_day, 12, 0)
    stop = datetime.datetime(year, end_month, end_day, 12, 0)

    print(f"Analyzing time series from {start.date()} to {stop.date()}")
    print(f"Slope angle: {slope_angle_ts}°")

    # Read all profiles from file
    profiles = readProfile.read_profile(pro_file, remove_soil=True)
    dates = pd.date_range(start, stop, freq='D')

    df_list = []
    missing_dates = []

    for ts in dates:
        if ts in profiles['data'].keys():
            prof = profiles['data'][ts]
            if (len(prof.keys()) == 0) or (len(prof['height']) == 0):
                # Empty profile - create placeholder
                df0 = pd.DataFrame(columns=['P_unstable','layer_top','density','hardness','graintype',
                                            'viscdefrate','rcflat','sphericity','grainsize',
                                            'penetrationdepth','slab_rhogs','HS'], index=[0])
                df0['HS'] = 0.0
            else:
                df0 = get_RF.create_RFprof(prof, slope_angle_ts, model)
                df0['HS'] = df0['layer_top'].iloc[-1]
            df0.insert(0, 'datetime', ts)
            df_list.append(df0)
        else:
            missing_dates.append(ts)

    if missing_dates:
        print(f"⚠ Warning: {len(missing_dates)} dates not found in profile file")

    if not df_list:
        print("⚠ No data found for the specified date range")
        print("Try adjusting the date range or check the .pro file contents")
    else:
        df_evo = pd.concat(df_list, ignore_index=True)
        
        print(f"✓ Analyzed {len(df_list)} profiles")
        
        # Plot
        fig, ax = plt.subplots(figsize=(10, 6))
        plt_RF.plot_evo_SP(df_evo, fig, ax, start, stop, var='P_unstable', colorbar=True, resolution='D')
        plt.title(f"Daily Evolution of P_unstable\n{os.path.basename(pro_file)} - Slope: {slope_angle_ts}°")
        plt.tight_layout()
        plt.show()

## Export Daily Summary CSV

Generate a CSV file with daily summary statistics for easy analysis.

In [ ]:
# @title Export Daily Summary CSV
# @markdown Generate a CSV file with daily summary statistics for easy analysis

# @markdown ## Export Parameters
export_year = 2017  # @param {type:"integer"}
export_start_month = 12  # @param {type:"integer", min:1, max:12}
export_start_day = 1  # @param {type:"integer", min:1, max:31}
export_end_month = 4  # @param {type:"integer", min:1, max:12}
export_end_day = 1  # @param {type:"integer", min:1, max:31}
export_slope_angle = 0  # @param {type:"number", min:0, max:90}

if not pro_file or not os.path.exists(pro_file):
    print("⚠ No .pro file selected. Please run the upload cell first.")
else:
    start = pd.Timestamp(export_year-1 if export_start_month == 12 else export_year, export_start_month, export_start_day, 12, 0)
    stop = pd.Timestamp(export_year, export_end_month, export_end_day, 12, 0)

    print(f"Generating daily summary from {start.date()} to {stop.date()}")
    print(f"Slope angle: {export_slope_angle}°")

    profiles = readProfile.read_profile(pro_file, remove_soil=True)
    rows = []

    for ts in pd.date_range(start, stop, freq='D'):
        prof = profiles['data'].get(ts)
        if not prof or len(prof.get('height', [])) == 0:
            continue
        
        try:
            dfi = get_RF.create_RFprof(prof, export_slope_angle, model)
            rows.append({
                'datetime': ts,
                'HS': float(dfi['layer_top'].iloc[-1]),
                'P_unstable_max': float(dfi['P_unstable'].max()),
                'z_Pmax': float(dfi.loc[dfi['P_unstable'].idxmax(), 'layer_top']),
                'P_unstable_mean': float(dfi['P_unstable'].mean())
            })
        except Exception as e:
            print(f"⚠ Error processing {ts}: {e}")
            continue

    if rows:
        out = pd.DataFrame(rows).sort_values('datetime')
        
        # Determine output path
        if os.path.exists("/content"):
            out_path = '/content/p_unstable_daily.csv'
        else:
            out_path = './p_unstable_daily.csv'
        
        out.to_csv(out_path, index=False)
        
        print(f"\n✓ Daily summary exported to: {out_path}")
        print(f"  Records: {len(out)}")
        print(f"\nPreview:")
        print(out.head(10).to_string(index=False))
        
        # Download in Colab
        try:
            from google.colab import files
            files.download(out_path)
            print("\n✓ File downloaded")
        except ImportError:
            print(f"\nFile saved at: {out_path}")
    else:
        print("⚠ No data found for the specified date range")